## Comparing error in least squares



In [10]:
import numpy as np
from scipy import linalg
import timeit

### Set up problem
Use van der Monde with three columns

In [11]:
rows = 8
c = np.ones(rows)
x = np.arange(2,4.1,0.2)
A = np.vander(x,rows)
y = np.matmul(A,c)
print(np.linalg.cond(A.T@A))

2.8888381436329173e+19


In [166]:
print(len(A),len(A.T), len(y))

11 8 11


### normal equations via matrix inverse

Given $A$ and $\mathbf{y}$, find the least sq

In [9]:
%%timeit

A2 = A.T @ A
y2 = A.T @ y
c_over = np.linalg.inv(A2)@(y2)

8.83 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
A2 = A.T @ A
y2 = A.T @ y
c_over = np.linalg.inv(A2)@(y2)
print("coefficients should be 1s:", [c for c in c_over])
y_over = A@c_over

coefficients should be 1s: [1.33203125, -3.8125, 26.0, -76.0, 176.0, -256.0, 224.0, -92.0]


### normal equations via Cholesky factorization

In [12]:
%%timeit
A2 = A.T @ A
y2 = A.T @ y
ch, low = linalg.cho_factor(A2)
c_over = linalg.cho_solve((ch, low), y2)


7.1 µs ± 56 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [4]:
ch, low = linalg.cho_factor(A2)
c_over = linalg.cho_solve((ch, low), y2)
print("coefficients should be 1s:", c_over)

coefficients should be 1s: [ 1.00058837  0.98760692  1.11086082  0.45419753  2.59686198 -1.77574037
  3.65379288 -0.07638327]


### QR

In [13]:
%%timeit
Q, R = np.linalg.qr(A, mode='reduced')
Q_T = np.transpose(Q)
product1 = np.dot(Q_T,y)
# default for solve_triangular is an upper triangular
c_over = linalg.solve_triangular(R, product1)

12.5 µs ± 80.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
Q, R = np.linalg.qr(A, mode='reduced')
Q_T = np.transpose(Q)
product1 = np.dot(Q_T,y)
# default for solve_triangular is an upper triangular
c_over = linalg.solve_triangular(R, product1)
print("coefficients should be 1s:", [c for c in c_over])

coefficients should be 1s: [1.000000000018532, 0.9999999996288799, 1.0000000031435226, 0.9999999854257074, 1.0000000398546254, 0.9999999359076287, 1.000000055899682, 0.9999999797170683]


In [17]:
%%timeit
Q, R = np.linalg.qr(A, mode='reduced')
Q_T = np.transpose(Q)
R_inv = np.linalg.inv(R)
product1 = np.dot(R_inv,Q_T)
c_over = np.dot(product1,y)

17.9 µs ± 759 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
Q, R = np.linalg.qr(A, mode='reduced')
Q_T = np.transpose(Q)
R_inv = np.linalg.inv(R)
product1 = np.dot(R_inv,Q_T)
c_over = np.dot(product1,y)
print("coefficients should be 1s:", c_over)

coefficients should be 1s: [1.         1.         1.         0.99999999 1.00000012 0.99999991
 1.00000015 0.99999994]


In [141]:
B = [[5,9],[1,7],[-3,-5],[1,5]]
Q, R = np.linalg.qr(B, mode='reduced')
print(Q)
print(R)

[[-0.83333333  0.16666667]
 [-0.16666667 -0.83333333]
 [ 0.5        -0.16666667]
 [-0.16666667 -0.5       ]]
[[ -6. -12.]
 [  0.  -6.]]


In [143]:
B@np.array([1,-1])

array([-4, -6,  2, -4])

In [7]:
print(R)

[[-2.25318595e+04 -5.90438580e+03 -1.55483115e+03 -4.11786319e+02
  -1.09793269e+02 -2.95081486e+01 -8.00692363e+00 -2.19803609e+00]
 [ 0.00000000e+00 -4.14083700e+02 -2.36639392e+02 -1.02639191e+02
  -4.01166244e+01 -1.49326921e+01 -5.43338002e+00 -1.96216573e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.86655995e+01 -1.75074665e+01
  -1.11279519e+01 -5.99972400e+00 -2.96737825e+00 -1.39782893e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.26125391e+00
   1.67616107e+00  1.41335378e+00  9.68388048e-01  5.89939068e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   8.72094850e-02  1.49604933e-01  1.55910472e-01  1.27975978e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -5.61392520e-03 -1.17591338e-02 -1.45146453e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.31072842e-04  8.16256162e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   

In [9]:
print(ch)

[[2.25318595e+04 5.90438580e+03 1.55483115e+03 4.11786319e+02
  1.09793269e+02 2.95081486e+01 8.00692363e+00 2.19803609e+00]
 [1.33036791e+08 4.14083700e+02 2.36639392e+02 1.02639191e+02
  4.01166244e+01 1.49326921e+01 5.43338002e+00 1.96216573e+00]
 [3.50332370e+07 9.27831148e+06 1.86655995e+01 1.75074665e+01
  1.11279519e+01 5.99972400e+00 2.96737825e+00 1.39782893e+00]
 [9.27831148e+06 2.47384651e+06 6.64873459e+05 1.26125391e+00
  1.67616107e+00 1.41335378e+00 9.68388048e-01 5.89939068e-01]
 [2.47384651e+06 6.64873459e+05 1.80410878e+05 4.95258403e+04
  8.72094852e-02 1.49604933e-01 1.55910473e-01 1.27975978e-01]
 [6.64873459e+05 1.80410878e+05 4.95258403e+04 1.37905539e+04
  3.90799200e+03 5.61392561e-03 1.17591327e-02 1.45146443e-02]
 [1.80410878e+05 4.95258403e+04 1.37905539e+04 3.90799200e+03
  1.13173280e+03 3.36600000e+02 3.31087186e-04 8.16263320e-04]
 [4.95258403e+04 1.37905539e+04 3.90799200e+03 1.13173280e+03
  3.36600000e+02 1.03400000e+02 3.30000000e+01 1.75665415e-05]]

### Error magnification

In [162]:
def error_magnification(c, c_calc, y, y_calc):
    forward_error = np.linalg.norm(c - c_calc)
    backward_error = np.linalg.norm(y - y_calc)
    print("forward",forward_error)
    print("backward",backward_error)
    error_mag_abs = forward_error/backward_error
    error_mag_rel = error_mag_abs*np.linalg.norm(y)/np.linalg.norm(c)
    return error_mag_abs, error_mag_rel